In [1]:
import os
import csv
import json
import tweepy
import sqlite3
import requests
import tweepy.api
import numpy as np
import pandas as pd
import seaborn as sns
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from statsmodels.api import OLS
from tweepy import OAuthHandler
from scipy.stats import pearsonr
from matplotlib import pyplot as plt
from timeit import default_timer as timer
from requests_oauthlib import OAuth1Session
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [3]:
def read_json(file_path):
    df = pd.read_json(file_path, lines=True)
    return df

# Usage
json_file = read_json('/content/tweet-json.json')

In [4]:
json_file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2354 entries, 0 to 2353
Data columns (total 31 columns):
 #   Column                         Non-Null Count  Dtype              
---  ------                         --------------  -----              
 0   created_at                     2354 non-null   datetime64[ns, UTC]
 1   id                             2354 non-null   int64              
 2   id_str                         2354 non-null   int64              
 3   full_text                      2354 non-null   object             
 4   truncated                      2354 non-null   bool               
 5   display_text_range             2354 non-null   object             
 6   entities                       2354 non-null   object             
 7   extended_entities              2073 non-null   object             
 8   source                         2354 non-null   object             
 9   in_reply_to_status_id          78 non-null     float64            
 10  in_reply_to_status_id_st

In [6]:
json_file.head(2)

,created_at,id,id_str,full_text,truncated,display_text_range,entities,extended_entities,source,in_reply_to_status_id,...,favorite_count,favorited,retweeted,possibly_sensitive,possibly_sensitive_appealable,lang,retweeted_status,quoted_status_id,quoted_status_id_str,quoted_status
0,2017-08-01 16:23:56+00:00,892420643555336193,892420643555336192,This is Phineas. He's a mystical boy. Only eve...,False,"[0, 85]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 892420639486877696, 'id_str'...","<a href=""http://twitter.com/download/iphone"" r...",NaN,...,39467,False,False,0.0,0.0,en,NaN,NaN,NaN,NaN
1,2017-08-01 00:17:27+00:00,892177421306343426,892177421306343424,This is Tilly. She's just checking pup on you....,False,"[0, 138]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 892177413194625024, 'id_str'...","<a href=""http://twitter.com/download/iphone"" r...",NaN,...,33819,False,False,0.0,0.0,en,NaN,NaN,NaN,NaN


In [7]:
print(json_file.isnull().sum())

created_at                          0
id                                  0
id_str                              0
full_text                           0
truncated                           0
display_text_range                  0
entities                            0
extended_entities                 281
source                              0
in_reply_to_status_id            2276
in_reply_to_status_id_str        2276
in_reply_to_user_id              2276
in_reply_to_user_id_str          2276
in_reply_to_screen_name          2276
user                                0
geo                              2354
coordinates                      2354
place                            2353
contributors                     2354
is_quote_status                     0
retweet_count                       0
favorite_count                      0
favorited                           0
retweeted                           0
possibly_sensitive                143
possibly_sensitive_appealable     143
lang        

In [9]:
twitter_archive = pd.read_csv('/content/twitter-archive-enhanced.csv')

In [10]:
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(url)
with open('/content/image_predictions.tsv', 'wb') as file:
    file.write(response.content)


In [11]:
image_predictions = pd.read_csv('/content/image_predictions.tsv', sep='\t')


In [12]:
def read_json(file_path):
    with open(file_path, 'r') as file:
        data = [json.loads(line) for line in file]
    df = pd.DataFrame(data)
    return df

tweet_json = read_json('/content/tweet-json.json')

In [13]:
tweet_data = tweet_json[['id', 'retweet_count', 'favorite_count']]
tweet_data.rename(columns={'id': 'tweet_id'}, inplace=True)


<ipython-input-13-3fe5d70f361f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweet_data.rename(columns={'id': 'tweet_id'}, inplace=True)


In [15]:
# Load datasets
twitter_archive = pd.read_csv('/content/twitter-archive-enhanced.csv')
image_predictions = pd.read_csv('/content/image_predictions.tsv', sep='\t')
tweet_json_2 = pd.read_json('/content/tweet-json.json')



# Visual Assessment
display(twitter_archive.head())
display(image_predictions.head())
display(tweet_json.head())

# Programmatic Assessment
# Twitter Archive Data
print(twitter_archive.info())
print(twitter_archive.describe())
print(twitter_archive.isnull().sum())
print(twitter_archive.duplicated().sum())

# Image Predictions Data
print(image_predictions.info())
print(image_predictions.describe())
print(image_predictions.isnull().sum())
print(image_predictions.duplicated().sum())

# Tweet JSON Data

print(tweet_json_1.info())
print(tweet_json_1.describe())
print(tweet_json_1.isnull().sum())
print(tweet_json_1.duplicated().sum())

# Documenting Quality Issues
quality_issues = [
    "Missing values in 'in_reply_to_status_id', 'in_reply_to_user_id', 'retweeted_status_id', 'retweeted_status_user_id', and 'retweeted_status_timestamp'.",
    "Incorrect data types for 'timestamp', 'in_reply_to_status_id', 'in_reply_to_user_id', 'retweeted_status_id', 'retweeted_status_user_id'.",
    "Erroneous dog names such as 'a', 'an', 'the'.",
    "Inconsistent dog stages (doggo, floofer, pupper, puppo) in a single column.",
    "Retweets and replies included in the dataset.",
    "Tweets without images.",
    "Rating numerators and denominators with incorrect values.",
    "Duplicate tweets."
]

# Documenting Tidiness Issues
tidiness_issues = [
    "Dog stages (doggo, floofer, pupper, puppo) should be in one column.",
    "Separate columns for rating numerator and denominator should be combined into one column."
]

# Displaying issues
print("Quality Issues:")
for issue in quality_issues:
    print(f"- {issue}")

print("\nTidiness Issues:")
for issue in tidiness_issues:
    print(f"- {issue}")


ValueError: Trailing data

In [16]:
# Load datasets
twitter_archive = pd.read_csv('/content/twitter-archive-enhanced.csv')
image_predictions = pd.read_csv('/content/image_predictions.tsv', sep='\t')
tweet_json_2 = pd.read_json('/content/tweet-json.json')

# Make copies of original data
twitter_archive_clean = twitter_archive.copy()
image_predictions_clean = image_predictions.copy()
tweet_json_clean = tweet_json.copy()

# Define, Code, Test Framework

# 1. Remove retweets and replies
# Define
twitter_archive_clean = twitter_archive_clean[twitter_archive_clean['retweeted_status_id'].isnull()]
twitter_archive_clean = twitter_archive_clean[twitter_archive_clean['in_reply_to_status_id'].isnull()]

# Test
print(twitter_archive_clean.info())

# 2. Remove tweets without images
# Define
twitter_archive_clean = twitter_archive_clean[twitter_archive_clean['expanded_urls'].notnull()]

# Test
print(twitter_archive_clean.info())

# 3. Convert timestamp to datetime
# Define
twitter_archive_clean['timestamp'] = pd.to_datetime(twitter_archive_clean['timestamp'])

# Test
print(twitter_archive_clean.info())

# 4. Correct dog names
# Define
twitter_archive_clean['name'] = twitter_archive_clean['name'].apply(lambda x: x if x.isalpha() else None)

# Test
print(twitter_archive_clean['name'].value_counts())

# 5. Combine dog stages into one column
# Define
twitter_archive_clean['dog_stage'] = twitter_archive_clean[['doggo', 'floofer', 'pupper', 'puppo']].apply(lambda x: ','.join(x.dropna().astype(str)), axis=1)
twitter_archive_clean['dog_stage'] = twitter_archive_clean['dog_stage'].replace('', None)

# Test
print(twitter_archive_clean['dog_stage'].value_counts())

# 6. Merge datasets
# Define
master_df = pd.merge(twitter_archive_clean, image_predictions_clean, on='tweet_id', how='inner')
master_df = pd.merge(master_df, tweet_json_clean, on='tweet_id', how='inner')

# Test
print(master_df.info())

# 7. Remove duplicates
# Define
master_df = master_df.drop_duplicates()

# Test
print(master_df.duplicated().sum())

# 8. Handle missing values
# Define
master_df = master_df.dropna(subset=['expanded_urls', 'rating_numerator', 'rating_denominator'])

# Test
print(master_df.isnull().sum())

# Tidiness Issues
# 1. Combine rating numerator and denominator
# Define
master_df['rating'] = master_df['rating_numerator'] / master_df['rating_denominator']

# Test
print(master_df['rating'].head())

# 2. Drop unnecessary columns
# Define
columns_to_drop = ['in_reply_to_status_id', 'in_reply_to_user_id', 'retweeted_status_id', 'retweeted_status_user_id', 'doggo', 'floofer', 'pupper', 'puppo']
master_df = master_df.drop(columns=columns_to_drop)

# Test
print(master_df.info())

# Save cleaned data
master_df.to_csv('/content/twitter-archive-enhanced.csv', index=False)


ValueError: Trailing data

In [ ]:
import pandas as pd
import sqlite3

# Load the cleaned master DataFrame
master_df = pd.read_csv('/content/twitter_archive_enhanced.csv')

# Store the cleaned master DataFrame in a CSV file
master_df.to_csv('/content/twitter_archive_master.csv', index=False)

# Store the cleaned master DataFrame in a SQLite database
conn = sqlite3.connect('twitter_archive_master.db')
master_df.to_sql('twitter_archive', conn, index=False, if_exists='replace')

# Close the connection
conn.close()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the cleaned master DataFrame
master_df = pd.read_csv('/content/twitter-archive-enhanced.csv')

# Insight 1: Distribution of ratings
plt.figure(figsize=(10, 6))
sns.histplot(master_df['rating'], bins=20, kde=True)
plt.title('Distribution of Dog Ratings')
plt.xlabel('Rating')
plt.ylabel('Frequency')
plt.show()

# Insight 2: Most common dog names
top_dog_names = master_df['name'].value_counts().head(10)
plt.figure(figsize=(10, 6))
sns.barplot(x=top_dog_names.index, y=top_dog_names.values, palette='viridis')
plt.title('Top 10 Most Common Dog Names')
plt.xlabel('Dog Name')
plt.ylabel('Count')
plt.show()

# Insight 3: Relationship between retweets and favorites
plt.figure(figsize=(10, 6))
sns.scatterplot(x='retweet_count', y='favorite_count', data=master_df, alpha=0.6)
plt.title('Relationship Between Retweets and Favorites')
plt.xlabel('Retweet Count')
plt.ylabel('Favorite Count')
plt.show()

# Documenting the data used
print("Data used for analysis and visualization:")
print(master_df.head())
